## previous traninaed model in d22b notebook

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers

train_dataset = image_dataset_from_directory(
    "cats_vs_dogs_small\\train",
    image_size=(180,180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    "cats_vs_dogs_small\\validation",
    image_size=(180,180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    "cats_vs_dogs_small\\test",
    image_size=(180,180),
    batch_size=32)

#convolution base
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
#setting convolution base to False to freeze the weights during training
conv_base.trainable = False 

data_augmentation = keras.Sequential(
    [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    ]
)

#our model
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs) #applying data augmentation
x = keras.applications.vgg16.preprocess_input(x) #applying value scaling
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(loss="binary_crossentropy",
             optimizer="rmsprop",
             metrics=["accuracy"])


history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Epoch 1/50
63/63 [==============================] - 42s 475ms/step - loss: 16.1557 - accuracy: 0.9020 - val_loss: 3.0563 - val_accuracy: 0.9740
Epoch 2/50
63/63 [==============================] - 26s 409ms/step - loss: 5.8072 - accuracy: 0.9590 - val_loss: 4.1970 - val_accuracy: 0.9690
Epoch 3/50
63/63 [==============================] - 26s 415ms/step - loss: 5.1584 - accuracy: 0.9540 - val_loss: 3.8796 - val_accuracy: 0.9770
Epoch 4/50
63/63 [==============================] - 26s 416ms/step - loss: 4.9104 - accuracy: 0.9615 - val_loss: 6.1690 - val_accuracy: 0.9710
Epoch 5/50
63/63 [==============================] - 27s 421ms/step - loss: 3.6770 - accuracy: 0.9755 - val_loss: 3.3626 - val_accuracy: 0.9750
Epoch 6/50
63/63 [==============================] - 27s 421ms/step - loss: 3.2950 - accuracy: 0.9720 - val_loss: 4.7018 - val_accuracy: 0.9730
Epoch 7/50
63/63 [

In [2]:
#Freezing all layers until the fourth from the list
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

#fine-tuning the model
model.compile(loss="binary_crossentropy",
             optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
             metrics=["accuracy"])
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks
)

Epoch 1/30
63/63 [==============================] - 36s 469ms/step - loss: 3.6409 - accuracy: 0.9735 - val_loss: 3.3292 - val_accuracy: 0.9800
Epoch 2/30
63/63 [==============================] - 28s 443ms/step - loss: 1.8272 - accuracy: 0.9770 - val_loss: 2.5546 - val_accuracy: 0.9830
Epoch 3/30
63/63 [==============================] - 28s 446ms/step - loss: 1.7530 - accuracy: 0.9740 - val_loss: 3.0309 - val_accuracy: 0.9810
Epoch 4/30
63/63 [==============================] - 28s 449ms/step - loss: 1.0800 - accuracy: 0.9830 - val_loss: 2.9912 - val_accuracy: 0.9790
Epoch 5/30
63/63 [==============================] - 29s 453ms/step - loss: 1.0161 - accuracy: 0.9865 - val_loss: 2.3937 - val_accuracy: 0.9800
Epoch 6/30
63/63 [==============================] - 29s 452ms/step - loss: 0.6099 - accuracy: 0.9910 - val_loss: 2.4818 - val_accuracy: 0.9820
Epoch 7/30
63/63 [==============================] - 29s 455ms/step - loss: 0.6613 - accuracy: 0.9860 - val_loss: 2.6330 - val_accuracy: 0.9820

In [3]:
test_model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 12s 174ms/step - loss: 2.5958 - accuracy: 0.9765
Test accuracy: 0.976
